In [15]:
import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv('inflation_annual_percent.csv')
df.fillna(0, inplace=True)
df=df.T
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

df.head()

country,Aruba,Afghanistan,Angola,Albania,Andorra,United Arab Emirates,Argentina,Armenia,American Samoa,Antigua and Barbuda,...,St. Vincent and the Grenadines,Venezuela,Virgin Islands (U.S.),Vietnam,Vanuatu,Samoa,Yemen,South Africa,Zambia,Zimbabwe
1961,0,0,0,0,0,0,20.3,0,0,0,...,2.5,2.01,0,0,0,0,0,1.35,−3.66,−2.04
1962,0,0,0,0,0,0,28.9,0,0,0,...,0.0537,0.665,0,0,0,0,0,0.383,2.09,0.47
1963,0,0,0,0,0,0,25.6,0,0,0,...,0.686,4.92,0,0,0,0,0,3.27,0.404,−2.35
1964,0,0,0,0,0,0,28.8,0,0,0,...,3.85,−0.366,0,0,0,0,0,1.99,4.08,6.14
1965,0,0,0,0,0,0,21.2,0,0,0,...,1.47,2.17,0,0,0,0,0,2.95,10.6,2.7


In [16]:
df.drop(df.columns.difference(['country', 'Pakistan']), axis=1, inplace=True)
df = df.replace('−', '-', regex=True)
df.head()

country,Pakistan
1961,3.65
1962,0.16
1963,-1.15
1964,4.49
1965,3.17


In [17]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model

# Define the input shape
input_shape = (60,)

# Define the encoding dimension
encoding_dim = 3

# Define the input layer
input_layer = Input(shape=input_shape)

# Define the encoding layer
encoding_layer = Dense(encoding_dim, activation='relu')(input_layer)

# Define the decoding layer
decoding_layer = Dense(60, activation='sigmoid')(encoding_layer)

# Define the autoencoder model
autoencoder = Model(input_layer, decoding_layer)

# Define the encoder model
encoder = Model(input_layer, encoding_layer)

# Define the decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Prepare the data
data = df.to_numpy().flatten().astype('float32')
data = np.reshape(data, (1, -1))  # Reshape to 2D tensor

# Train the autoencoder model
autoencoder.fit(data, data, epochs=50)

# Use the encoder to get the encoded representation of the data
encoded_data = encoder.predict(data)

# Use the decoder to get the reconstructed data from the encoded data
decoded_data = decoder.predict(encoded_data)

# Print the reconstructed data
print(decoded_data)


Epoch 1/50
1/1 [==============================] - 1s 730ms/step - loss: 0.2801
Epoch 2/50
1/1 [==============================] - 0s 4ms/step - loss: -0.0222
Epoch 3/50
1/1 [==============================] - 0s 4ms/step - loss: -0.3213
Epoch 4/50
1/1 [==============================] - 0s 4ms/step - loss: -0.6172
Epoch 5/50
1/1 [==============================] - 0s 4ms/step - loss: -0.9105
Epoch 6/50
1/1 [==============================] - 0s 3ms/step - loss: -1.2023
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: -1.4947
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: -1.7911
Epoch 9/50
1/1 [==============================] - 0s 3ms/step - loss: -2.0953
Epoch 10/50
1/1 [==============================] - 0s 4ms/step - loss: -2.4111
Epoch 11/50
1/1 [==============================] - 0s 4ms/step - loss: -2.7418
Epoch 12/50
1/1 [==============================] - 0s 4ms/step - loss: -3.0894
Epoch 13/50
1/1 [==============================] - 0s 3ms/st

In [18]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model

# Load the CSV file
df = pd.read_csv('inflation_annual_percent.csv')
df.fillna(0, inplace=True)

# Transpose the dataframe
df = df.T

# Set the first row as the column headers
df.columns = df.iloc[0]
df = df[1:]

# Replace any negative signs
df = df.replace('−', '-', regex=True)
df.replace('k', '000', inplace=True, regex=True) # remove the 'k' character

df.replace('µ', '', inplace=True, regex=True) # remove the 'k' character

# Define the input shape
input_shape = (60,)

# Define the encoding dimension
encoding_dim = 3

# Define the input layer
input_layer = Input(shape=input_shape)

# Define the encoding layer
encoding_layer = Dense(encoding_dim, activation='relu')(input_layer)

# Define the decoding layer
decoding_layer = Dense(60, activation='sigmoid')(encoding_layer)

# Define the autoencoder model
autoencoder = Model(input_layer, decoding_layer)

# Define the encoder model
encoder = Model(input_layer, encoding_layer)

# Define the decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
# Loop through each column in the dataframe
for column in df:
    # Prepare the data for the current column
    data = df[column].to_numpy().astype('float32')
    data = np.reshape(data, (1, -1))  # Reshape to 2D tensor

    # Train the autoencoder model
    autoencoder.fit(data, data, epochs=50)

    # Use the encoder to get the encoded representation of the data
    encoded_data = encoder.predict(data)

    # Use the decoder to get the reconstructed data from the encoded data
    decoded_data = decoder.predict(encoded_data)

    # Predict the next 10 values
    for i in range(10):
        # Get the last 60 values of the data
        input_data = data[:, -60:]

        # Use the encoder to get the encoded representation of the data
        encoded_input = encoder.predict(input_data)

        # Use the decoder to get the reconstructed data from the encoded data
        predicted_data = decoder.predict(encoded_input)

        # Append the predicted value to the input data
        input_data = np.append(input_data, predicted_data[:, -1:], axis=1)

        # Update the data with the predicted value
        data = np.append(data, predicted_data[:, -1:], axis=1)

        # Print the predicted value
        print(f'{column}_prediction_{i+1}: {predicted_data[:, -1][0]}')

    # Save the predicted values to a separate CSV file
    predicted_df = pd.DataFrame(data[:, -10:], columns=[f'{i+2020}' for i in range(10)])
    predicted_df.to_csv(f'{column}_predictions.csv', index=False)



Epoch 1/50
1/1 [==============================] - 1s 518ms/step - loss: 0.6587
Epoch 2/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6368
Epoch 3/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6151
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: 0.5935
Epoch 5/50
1/1 [==============================] - 0s 4ms/step - loss: 0.5721
Epoch 6/50
1/1 [==============================] - 0s 4ms/step - loss: 0.5507
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: 0.5295
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: 0.5084
Epoch 9/50
1/1 [==============================] - 0s 5ms/step - loss: 0.4874
Epoch 10/50
1/1 [==============================] - 0s 5ms/step - loss: 0.4665
Epoch 11/50
1/1 [==============================] - 0s 5ms/step - loss: 0.4457
Epoch 12/50
1/1 [==============================] - 0s 4ms/step - loss: 0.4249
Epoch 13/50
1/1 [==============================] - 0s 4ms/step - loss: 